In [1]:
#For Truth experiments
from truth_accountant import *

In [2]:
sigma = 2; q = mp.mpf('0.01'); comps = 3000; eps = 2; delta = 2e-12
eps_lower = 1.2945489807461993e-12; eps_upper = 2.61624568229506e-12

Running with CLT bounds from saddlepoint yeilds a result, albeit a slow one:

Took more than 45 minutes. Do not have time for this

In [3]:
%%time
compute_epsilon_mp(sigma, q, comps, eps, eps_lower = eps_lower, eps_upper = eps_upper)

KeyboardInterrupt: 

Running compute epsilon mp naivly yields a numerical error:

In [3]:
%%time
compute_epsilon_mp(sigma, q, comps, eps)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jug103\spa_env\lib\site-packages\IPython\core\magics\execution.py", line 1308, in time
    out = eval(code, glob, local_ns)
  File "<timed eval>", line 1, in <module>
  File "C:\Users\jug103\Documents\saddlepoint_accountant\truth_accountant.py", line 63, in compute_epsilon_mp
  File "C:\Users\jug103\spa_env\lib\site-packages\mpmath\calculus\optimization.py", line 985, in findroot
    raise ValueError('Could not find root within given tolerance. '
ValueError: Could not find root within given tolerance. (3.5519724459270825626 > 2.16840434497100886801e-19)
Try another starting point or tweak arguments.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\jug103\spa_env\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\jug103\spa_env\lib\site-packages\IPython\core\ultratb.p

## NO CHANGE IN DOING REAL PART FIRST OR SECOND. I THINK IT IS MARGINALLY FASTER AND MAKES MORE SENSE, SO WE WILL DO IT. 

For some reason doing the 

Doing the original integral, i.e taking the complex integral, then taking the real part at the end 

In [3]:
%%time
compute_delta_mp(sigma, q, comps, eps)

(1.5009066223813e-11 + 0.0j)
CPU times: total: 3min 28s
Wall time: 3min 28s


mpf('1.9557570709204119e-12')

Doing the more "intelligent" route of taking the real part first, then evaluating only the real integral:

In [4]:
%%time
compute_real_delta_mp(sigma, q, comps, eps)

CPU times: total: 3min 25s
Wall time: 3min 25s


mpf('1.9557570709204119e-12')

Try doing the mpmath integral in PLRV space. May be fast or more accurate, idk. 

In [3]:
%%time
compute_plrv_delta_mp(sigma, q, comps, eps)

CPU times: total: 22min 2s
Wall time: 22min 3s


mpf('-6.7286420627037462e+24940')

## CODE FOR PLRV PDF INTEGRAL

IT IS WRONG SOMEHOW. NOT SURE WHY. DO NOT HAVE TIME TO CHECK IT

In [ ]:
def log_p_mp(sigma, q, z):
    '''
    Log density of subsampled Gaussian PLRV at point z
    '''

    if mp.almosteq(z, mp.log(1-q)):
        return -mp.inf
    logA = mp.log(sigma) + (mp.log(q) - mp.log(2*mp.pi))/2 - 1/(8*sigma*sigma)
    logg = mp.log( mp.exp(z) - 1 - q)
    return logA + 2*z - mp.mpf('3/2')*logg - sigma*sigma/(2)*(logg-mp.log(q)) * (logg-mp.log(q))

def new_compute_MGF(sigma,q,t):
    
    plrv_pdf = lambda l: log_p_mp(sigma,q,l)
    integrand = lambda l: mp.exp(t * l + plrv_pdf(l))
    bounds = (-mp.inf, mp.inf)
    integral, _ = mp.quad( integrand, bounds, error = True, maxdegree = 8 )
    return integral

def compute_plrv_delta_mp(sigma, q, comp, eps, a = mp.mpf('0.1')):

    def mgf(t): return new_compute_MGF(sigma,q, mp.mpc(a,t))

    def g(t): return mp.power( mp.mpc(a + a*a - t*t, 2*a*t + t), -1)

    bounds = (-mp.inf, mp.inf)
    def integrand(t): return mp.re(mp.expj(-t*eps) * \
                                   mp.power(mgf(t),comp) * g(t))
    integral, _ = mp.quad(integrand, bounds, error= True, maxdegree = 8)
    return mp.exp(-a*eps) / two_pi * integral
